In [1]:
import pandas as pd
import numpy as np

df = pd.read_csv('data/logs.csv')
display(df.shape)
df.head()

(2065516, 15)

,timeRead,timeDelete,id,totalTime,timestamp,serverType,clientTotalTime,requestSize,frequency,instanceType,freeMem,timeWrite,databaseType,totalMem,memUsage
0,-1.0,-1.0,--4qETfhcD,-1.0,-1.0,java,-1.0,50.0,15.0,b1,-1.0,-1.0,nosql,-1.0,-1.0
1,-1.0,-1.0,--Lo0QBj3fC,-1.0,-1.0,java,-1.0,200.0,40.0,b1,-1.0,-1.0,nosql,-1.0,-1.0
2,-1.0,-1.0,--P-Z58M2G,-1.0,-1.0,java,-1.0,1.0,15.0,b1,-1.0,-1.0,nosql,-1.0,-1.0
3,-1.0,-1.0,--nUeIfVXJ,-1.0,-1.0,java,-1.0,100.0,15.0,b1,-1.0,-1.0,nosql,-1.0,-1.0
4,-1.0,-1.0,--od3lhTv,-1.0,-1.0,java,-1.0,100.0,15.0,b1,-1.0,-1.0,nosql,-1.0,-1.0


In [2]:
servers = sorted(['java', 'nodejs'])
databases = sorted(['nosql', 'sql'])
instances = sorted(['b1', 'b2', 'b4', 'b8'])

combinations = [(servers[0], databases[0]), (servers[1], databases[0]), (servers[0], databases[1]), (servers[1], databases[1])]

sizes = sorted(list(df['requestSize'].unique()))
frequencies = sorted(list(df['frequency'].unique()))
types = np.transpose([np.tile(sizes, len(frequencies)), np.repeat(frequencies, len(sizes))])

In [3]:
types = np.transpose([np.tile(sizes, len(frequencies)), np.repeat(frequencies, len(sizes))])
df.dropna(inplace=True)
display(df.shape)

(2065516, 15)

In [4]:
cols = df.dtypes.to_dict()

query = None

for col in cols:
    if cols[col] == 'float64':
        temp_query = (df[col] > 0)
    else:
        temp_query = (df[col] != '')
        
    temp_query = temp_query & (df[col].notnull())
    
    if query is None:
        query = temp_query
        
    else:
        query = query & temp_query
        
        
clean_df = df.loc[query]
clean_df.to_csv('data/clean.csv', index=False)
display(clean_df.shape)
clean_df.head()

(1548823, 15)

,timeRead,timeDelete,id,totalTime,timestamp,serverType,clientTotalTime,requestSize,frequency,instanceType,freeMem,timeWrite,databaseType,totalMem,memUsage
218499,3.0,81.0,VWo2CJrUe,236.0,1.585877e+12,java,425.0,1.0,1.0,b1,1.887261e+09,8.0,sql,2.147484e+09,0.121176
218500,3.0,91.0,Ge9D-u1oz,299.0,1.585877e+12,java,489.0,1.0,1.0,b1,1.887121e+09,9.0,sql,2.147484e+09,0.121241
218501,3.0,7.0,W789KeB31,154.0,1.585877e+12,java,292.0,1.0,1.0,b1,1.886994e+09,10.0,sql,2.147484e+09,0.121300
218502,86.0,8.0,YW4UJ3m4N,255.0,1.585877e+12,java,448.0,1.0,1.0,b1,1.886646e+09,10.0,sql,2.147484e+09,0.121462
218503,4.0,6.0,RRAbhpXkJ,152.0,1.585877e+12,java,288.0,1.0,1.0,b1,1.886482e+09,9.0,sql,2.147484e+09,0.121538


In [7]:
counts = df.groupby(['instanceType', 'databaseType', 'serverType', 'requestSize', 'frequency']).count().reset_index()
counts.drop(['freeMem', 'timeWrite', 'totalMem', 'timeRead', 'timeDelete', 'totalTime', 'timestamp', 'clientTotalTime', 'memUsage'], axis=1, inplace=True)
counts.rename(columns={'id': 'count'}, inplace=True)
counts.sort_values(by=['count'], ascending=True, inplace=True)
counts.to_csv('data/counts.csv', index=False)
counts

(1058, 6)

,instanceType,databaseType,serverType,requestSize,frequency,count
261,b1,sql,nodejs,200.0,60.0,51
127,b1,nosql,nodejs,200.0,60.0,53
397,b2,nosql,nodejs,200.0,60.0,56
177,b1,sql,java,100.0,2.0,57
532,b2,sql,nodejs,200.0,60.0,61
...,...,...,...,...,...,...
736,b4,sql,java,200.0,60.0,18325
865,b8,nosql,java,200.0,60.0,18329
969,b8,sql,java,50.0,60.0,18383
979,b8,sql,java,100.0,60.0,18389
